In [ ]:
import torch
import zetta_utils
from zetta_utils.parsing import cue
from zetta_utils import builder

In [ ]:
print(torch.__version__)

In [ ]:
zetta_utils.load_all_modules()

In [ ]:
spec = '''
FM: 32

"model": {
    "@type": "torch.nn.Sequential",
    "modules": [
        {
            "@type": "ConvBlock",
            "num_channels": [1, FM],
            "kernel_sizes": [5, 5],
            "activate_last": true,
        },
        {
            "@type": "UNet",
            "@version": "0.0.1",
            "list_num_channels": [
                [FM, FM, FM, FM*2],
                [FM*2, FM*2, FM*2, FM*4],
                [FM*4, FM*4, FM*4, FM*8],
                [FM*8, FM*8, FM*8, FM*8],
                [FM*4, FM*4, FM*4, FM*4],
                [FM*2, FM*2, FM*2, FM*2],
                [FM, FM, FM, FM],
            ],
            "downsample": {
                "@type": "torch.nn.MaxPool2d",
                "@mode": "partial",
                "kernel_size": 2,
            },
            "upsample": {
                "@type": "UpConv",
                "@mode": "partial",
                "kernel_size": 3,
                "upsampler": {
                    "@type": "torch.nn.Upsample",
                    "@mode": "partial",
                    "scale_factor": 2,
                    "mode": "nearest",
                    "align_corners": null,
                },
                "conv": {
                    "@type": "torch.nn.Conv2d",
                    "@mode": "partial",
                    "padding": 1,
                },
            },
            "activate_last": true,
            "kernel_sizes": [3, 3],
            "unet_skip_mode": "sum",
            "skips": {"0": 3},
        },
        {
            "@type": "torch.nn.Conv2d",
            "in_channels": FM,
            "out_channels": 1,
            "kernel_size": 1,
        },
        {"@type": "torch.nn.Tanh"},
    ],
}
'''

In [ ]:
result = builder.build(cue.loads(spec))

In [ ]:
model = result['model']

In [ ]:
x = torch.randn([1, 1, 1024, 1024])
y = model(x)

In [ ]:
torch.onnx.export(
    model,
    (x, {}),
    "encoder.onnx",
    verbose=False,
    export_params=True,
    opset_version=11,
    input_names=["input"],
    output_names=["output"],
)